In [ ]:
'''
功能外部定义：
1.先确认是否要预处理脏数据
2.是否自动迭代出class的标签图片数据 默认是False 参数klass, autoklass
3.是否上按照自己提供的类目文件名去查找 标签图片数据 默认值是classes = '0_pass', '1_infringement'

功能内部实现：
1.判断传入路径是否合法
2.将要迭代的classes类(传入的classes结构)变成tuple节省内存和位置标准化
3.按照函数传入参数值来分别判断需要迭代的图片是否要自动话还是给定( 参照外部定义2 )来迭代出 file_list
4.创建4个list 分别是 bad_size, verify, convert_error, total_bad;并把满足各个条件的路径添加到list同时删除
   file_list中这些满足4个list的坏条件, [最终方法返回]良好的file_list 和4个坏的图片路径bad_size, verify...etc
5.详细流程说明：
    I    通过尾部名称迭代所有的.jpg或.jpeg文件，但因为图片内部字节码并不一定是真正的jpeg格式,故引入II
    II   通过I 迭代出来的所有图片路径索引, 分别迭代每个图片, 并按照4个list的流程去添加其列表,以及创建错
         误类的图片文件夹.并把文件夹添加到所给定的合法父路径中,最后将错误图片文件分别移至这些文件夹
    III  4个list 流程：
        bad_size：图片字节小于1则添加入list以及创建NONE_SIZE文件夹(不小就不创建),add到文件夹;continue
        *middle_method : 关键字(  img_format ,可用ctrl+F查找 ) 用imghdr的外部扩展包的 str( imghdr.what(
                                    文件迭代 ) )方法来获取自定义的图片结构,默认是jpeg, png, git 当满足其中任意结构则
                                    continue继续迭代，直到不满足进入verify检查；continue
        verify：检查图片是否可以打开或破损(通过bad_size, middle_method法判定结束, 及不满足前2者要求所
                                    执行 ).在verify的大判断内进行continue
        convert_error：**当[不]满足前2者,及图片字节[不]小于1,格式[不]满足jpeg, png, git;却能***[满足]verify
                                   及图片可正确打开,则执行图片的细致辨识；
                                   细致辨识：
                                       A: img_open.size=>'SMALL_SIZE'
                                           通过PIL的Image.open函数来读取图片, 后调用方法 .size赋值给变量 x, y 及图片w,h
                                           若图片长宽都任意一边小于299 则将此图片路径从总路径file_list中remove出去同时
                                           创建'SMALL_SIZE'的文件夹,用shutil.move方法把路径移动到'SMALL_SIZE' continue
                                      B:  通过A的continue方式过滤掉小于299的图片directory,所剩下的路径通过 *&open_
                                           format查看尾部名虽然为.jpg但内部字节不为jpg,png,gif,大小大于1个字节,能正常打
                                           开又双边都大于299的图片的格式。
                                           若此格式 != 'JPEG':
                                                  1. 通过piexif.remove(dirs)方法去除EXIF大文件的陈杂信息所导致的错误
                                                  2. 创建 'OTHER_MODE_CONVERT'文件夹,同时为了保证
                                                  3. Dhgate的文件能保有某些目的拥有的元名称不被删除,故用了tmp_img = 
                                                   other_mode_dir +os.sep+os.path.basename(dirs).split('.')[0]+ '.jpg'来保
                                                   全测试图片既可以拥有basename又可以有(纠正扩展名('.jpg'),其实大目录
                                                   迭代已经按照扩展名迭代,所以这里有些多余后期还需要改善大目录迭代的
                                                   方式,比方说用re[os.path.join(r,f) for r,ds,fs in os.walk(path) for f in fs if 
                                                   re.match(r".*\.(%s|%s)"%('jpg','jpeg'), f)]
                                                   )
                                                   if 此格式 != 'JPEG' 但mode == 'RGB' or mode == 'RGBX' 的话, 通过PIL的
                                                   Image.open()中的convert方法将图片转化成'RGB'同时移动到'OTHER_M
                                                   ODE'文件夹
                                                   else 就把此图片通过PIL的Image.new("RGB", 此图片大小, (白, 白, 白))和
                                                           image_new.paste(img_open, img_open)粘贴图像并保存到'OTHER_
                                                           MODE_CONVERT文件夹下
                                                   if  'OTHER_MODE'文件夹存在且大小不为0：
                                                           就将这些convert好的图片返回当前路径
                                                   else 就说明图片转化失败了,失败的路径添加到convert_error
                
                            vertify的模块中continue                    
         total_bad：在exception下的错误来添加到这个list中

6.  如果迭代的父路径的链接小于20个或者大于134M则报错        

附注：
NO.1 : *middle_method 是解释说明的文本自定义名，并非对应code的真实变量名，但在code中的具体实现是通过
                                        img_format的变量名实现的。
NO.2 : **当[不]满足：   当前判定是通过前面的已有判定中if 的continue方式过滤掉（满足跳过继续循环）
NO.3: 却能***[满足]:     当前判定是通过当前的if not 语句及if not 的方式, verify函数同一般函数一样返回默认None
                                       （没有返回值）的方式表述图片是合法。所以用if not None：就是True及在图片合法的情形
                                       下...etc
NO.4 : *&open_format ：为了在判别前能打印出图片格式，却不想在每张图片都调用Image.open(dirs)或 .format
                                            方法
                                           起初的open_format会在简单就能辨识图片好坏的条件下就赋值了open_format=字符串
                                           （None）来方便简单判别的的print调用下的传参.
by Geruigang.'''

In [38]:
class FLAGS():
    pass
FLAGS.min_file_size = 1
FLAGS.check_image_format = True
FLAGS.MAX_NUM_IMAGES_PER_CLASS = 2 ** 27 - 1  # ~134M
base_path = '/home/search/apps/jupyter/opt/samples/dhgate_full'
val_path = '/home/search/apps/jupyter/opt/samples/dhgate_full/2_validate'

if FLAGS.check_image_format:
    def process(target_path, klass, autoklass=False,):
        import logging, imghdr, os, shutil, piexif, cv2
        from PIL import Image
        from tqdm import tqdm_notebook as tqdm
        from collections import deque,OrderedDict
        result = OrderedDict()
        path = target_path
        if not os.path.exists(path):
            logging.error("Image directory '" + path + "' not found.")
            return None
        if autoklass is False:
            klass = tuple(klass) if klass is not tuple else klass
            file_list = [os.path.join(r,f)for label in klass for r,ds, fs in sorted(os.walk(os.path.join(path,label))) 
                        for f in fs if f.lower().endswith('jpg') or f.lower().endswith('jpeg')]
        else:
            #klass = os.listdir(path)
            file_list = [os.path.join(r,f)for r,ds, fs in sorted(os.walk(path)) 
                        for f in fs if f.lower().endswith('jpg') or f.lower().endswith('jpeg')]
        #     classes_label = len(klass)
        #     for cartegori in pic_dirs:
        #         if classes[0] in cartegori:a
        #             pass_0.append(cartegori)
        #         elif classes[1] in cartegori:
        #             infringement_1.append(cartegori)

        #     logging.info(f'''0_pass的数量有 {len(pass_0)} 张,
        #     \t1_fringement的数量有 {len(infringement_1)} 张,
        #     \t\t总共属于{len(classes)}个类别,合计有{len(infringement_1)+len(pass_0)} 张.''')

        none_size, verify, convert_error, total_bad = [], [], [], []
        logging.info('start check image format')
        for num, dirs in enumerate(tqdm(file_list)):
            try:
                open_format = 'None'
                getsize = os.path.getsize(dirs)
                if getsize < FLAGS.min_file_size:   
                    none_size.append(dirs)
                    none_size_dir = os.path.join(path,"NONE_SIZE")
                    if not os.exists(none_size_dir): 
                        os.makedirs(none_size_dir)
                        logging.info('Make the the %s dircetory that DHgate NONE_SIZE file in your path directory '%none_size_dir)
                    file_list.remove(img)
                    logging.info(f'Remove {dirs}')
                    logging.warning('%s too small file size %d ,the NO.%s removed' % (dirs, getsize,num))
                    continue#直接略过开下个dirs
                    # try:
                    #    img_open = Image.open(dirs)
                    #    call_verify = img_open.verify()
                    #   if call_verify is None:
                    #       shutil.rmtree(dirs)
                    #       #Additionally, you might want to also check that the [Dhgate images ]are actually a JPG by looking at the image format
                    #       #通过查看图像格式来检查图像实际上是JPG：
                    #   excepti ...
                img_format = str(imghdr.what(dirs)) 
                if img_format == 'jpeg' or img_format == 'png' or img_format == 'gif':
                    continue
                else:
                    #cv_check = cv2.imread(dirs)
                    img_open = Image.open(dirs)
                    x, y = img_open.size
                    mode = img_open.mode
                    open_format = img_open.format
                    call_verify = img_open.verify()
                    if not call_verify:
                        if x < 299 or y < 299:
                            file_list.remove(dirs)
                            #os.remove(img)
                            small_size_dir  = os.path.join(path, 'SMALL_SIZE')
                            if not os.path.exists(small_size_dir):
                                os.makedirs(small_size_dir)
                                logging.info('Make the the %s dircetory that DHgate SMALL_SIZE file in your path directory '%small_size_dir)
                            shutil.move(dirs, small_size_dir)
                            logging.warning(
                            '%s too small img size %dx%d , the NO.%s removed to SMALL_SIZE file' % (dirs, img_open.size[0], img_open.size[1], num))
                            continue
                            #                 else:
                            #                     verify.append(dirs)
                            #                     logging.warning('%s check verify the NO.%s Invalid image'%(dirs,num))

                        if open_format != 'JPEG':
                            piexif.remove(dirs)
                            other_mode_dir = os.path.join(path, 'OTHER_MODE_CONVERT')
                            if not os.path.exists(other_mode_dir):
                                os.makedirs(other_mode_dir)
                                logging.info('Make the %s dircetory that DHgate OTHER_MODE_CONVERT file in your path directory '%other_mode_dir)
                            tmp_img = other_mode_dir +os.sep+os.path.basename(dirs).split('.')[0]+ '.jpg'#或者直接str（num)
                            if mode == 'RGB' or mode == 'RGBX':
                                img_open.convert('RGB')
                                img_open.save(tmp_img)
                            else:
                                image_new = Image.new("RGB", img_open.size, (255, 255, 255))
                                image_new.paste(img_open, img_open)
                                image_new.save(tmp_img)

                            if os.path.exists(tmp_img) and os.path.getsize(tmp_img) > 0:
                                shutil.move(tmp_img, dirs)
                                logging.info('%s %s convert to the NO.%s jpeg success' % (dirs, open_format, num))
                            else:
                                logging.warning('%s %s convert to the NO.%s jpeg error' % (dirs, open_format, num))
                                file_list.remove(dirs)
                                convert_error.append(dirs)
                        continue#verity
                    else:
                        verify.append(dirs)
                        logging.warning('%s check verify the NO.%s Invalid image'%(dirs,num))

            except (IOError, ValueError) as err:
                #file_list.remove(dirs)
                #os.remove(img)
                bad_path = os.path.join(path, open_format)
                if not os.path.exists(bad_path):
                    os.makedirs(bad_path)
                    logging.info('Make the %s directory that DHgate the %s file in your path directory '%(bad_path, open_format))
                total_bad.append(dirs)
                shutil.move(dirs, bad_path)
                logging.warning('%s %s error : %s, removed' % (dirs, open_format, err, num))
                continue

        if len(file_list) < 20:
            logging.warning('Folder ' + ' only has ' + str(len(file_list))+ ' less than 20 images, skiped')
            #continue 在labbel中迭代才需要
        elif len(file_list) > FLAGS.MAX_NUM_IMAGES_PER_CLASS:
            tf.logging.warning(
                'Folder {} has more than {} images. Some images will '
                'never be selected.'.format(os.path.join(image_dir, label), MAX_NUM_IMAGES_PER_CLASS))
        return   file_list, none_size, verify, convert_error, total_bad
       
path = '/home/search/apps/jupyter/opt/samples/dhgate_full'
classes = '0_pass', '1_infringement'
file_list, none_size, verify, convert_error, total_bad = process(path,classes)
print(f'修复后的图片directory 有 {len(file_list)}张 ,保存在变量名 file_list 当中;',
      f'没有字节的图片有{len(none_size)}张,保存在变量名 none_size 当中;',
      f'\t不能打开的图片有{len(verify)}张.保存在变量名 verify当中;',
      f'\t\t转换失败的有{len(convert_error)}张,保存在变量名 convert_error 当中;',
      f'\t\t\t 最终预处理不能的图片有{len(total_bad)}张,保存在变量名 total_bad 当中.',sep='\n\t')
#from collections import deque,OrderedDict
pass_0, infringement_1 = [], []#deque(), deque()
for cartegori in file_list:
    if classes[0] in cartegori:
        pass_0.append(cartegori)
    elif classes[1] in cartegori:
        infringement_1.append(cartegori)
print()
print(f'0_pass的数量有 {len(pass_0)} 张;',
      f'1_fringement的数量有 {len(infringement_1)} 张;',
      f'\t总共属于2类别的有{len(infringement_1)+len(pass_0)}张.',
     sep='\n\t')


修复后的图片directory 有 762750张 ,保存在变量名 file_list 当中;
	没有字节的图片有0张,保存在变量名 none_size 当中;
		不能打开的图片有0张.保存在变量名 verify当中;
			转换失败的有0张,保存在变量名 convert_error 当中;
				 最终预处理不能的图片有0张,保存在变量名 total_bad 当中.

0_pass的数量有 381375 张;
	1_fringement的数量有 381375 张;
		总共属于2类别的有762750张.


In [30]:
len(pass_0)

381375

In [12]:
import numpy as np
import re,logging,os,shutil
from tqdm import tqdm_notebook as tqdm

classes1,classes2 = '0_pass', '1_infringement'
path = '/home/search/apps/jupyter/opt/samples/dhgate_full'
np.random.seed(20180123)
surplus_num = len(pass_0)-len(infringement_1)
logging.info('the surplus class have %s pictures'%surplus_num)#numpy array 不能用%d

surplus_random = np.random.choice(pass_0,surplus_num,replace=False)
assert len(surplus_random)==len(set(surplus_random))
logging.info('random to choice surplus  %s pictures'%surplus_random)#numpy array 不能用%d
mv_file_name = '4_surplus_data_0_pass'#4_surplus_data_0_pass
for i in tqdm(surplus_random):
    pass_0.remove(i)
len(pass_0)        

423751

In [14]:
if 1e-5:
    print(2)

2


In [15]:
'''注意这个地方，即便2个文件个数相等得到差值为0,通过np的choice 也会得到array的返回值，这并不是我们想要的'''
import numpy as np;np.random.choice(pass_0,0,replace=False)

array([], dtype='<U83')

In [32]:
import numpy as np
import re,logging,os,shutil
from tqdm import tqdm_notebook as tqdm

classes1,classes2 = '0_pass', '1_infringement'
path = '/home/search/apps/jupyter/opt/samples/dhgate_full'
np.random.seed(20180123)
surplus_num = len(pass_0)-len(infringement_1)
logging.info('the surplus class have %s pictures'%surplus_num)#numpy array 不能用%d
if surplus_num:
    surplus_random = np.random.choice(pass_0,surplus_num,replace=False)
    assert len(surplus_random)==len(set(surplus_random))
    logging.info('random to choice surplus  %s pictures'%surplus_random)#numpy array 不能用%d
    mv_file_name = '4_surplus_data_0_pass'#4_surplus_data_0_pass
    for i in tqdm(surplus_random):
        try:
            #pass_0.remove(i)#这个会严重拖延速度因为对应元素找list会是O（N）的复杂度
            #如果改成deque用线程也是没办法对应元素删除
            #方案1 用shuffle 然后直接把做切片，删除切片区间的路径
            #方案2 通过用label 一边迭代一边删除，但这个方法是不正确的，因为在没有迭代出label
            #类的总个数以及和其他类别的个数是无法知道要多了多少个path需要移动
            re_name  = re.sub(r'(%s|%s)'%(classes1,classes2),mv_file_name,i)
            re_name_file = os.path.split(re_name)[0]
            #surplus_file = os.path.join(path, mv_file_name)
            if not os.path.exists(re_name_file):
                os.mkdir(re_name_file)#尽量不要用mkdir 因为迭代不深,发现没有路径就报错了
            shutil.move(i,re_name_file)
            continue
            print('surplus file %s removed done'%surplus_num)
            print('0_pass have %s pictures'%pass_0)
        except Exception as e:
            logging.warning(e)

In [34]:
path = '/home/search/apps/jupyter/opt/samples/dhgate_full'
pass_0=[os.path.join(r,f) for r,ds,fs in os.walk(os.path.join(path,'0_pass')) for f in fs if f.lower().endswith('jpg')or f.lower().endswith('jpeg')]
infringement_1=[os.path.join(r,f) for r,ds,fs in os.walk(os.path.join(path,'1_infringement')) for f in fs if f.lower().endswith('jpg')or f.lower().endswith('jpeg')]
len(pass_0)
assert len(pass_0)==len(infringement_1)

In [33]:
len(infringement_1)

381375

In [ ]:
''' 严重警告：

17% 14766/84752 [00:10<00:51, 1353.33it/s]

WARNING:root:[Errno 2] No such file or directory: '/home/search/apps/jupyter/opt/samples/dhgate_full/2_validate0_pass/135010008'
WARNING:root:[Errno 2] No such file or directory: '/home/search/apps/jupyter/opt/samples/dhgate_full/2_validate0_pass/104006010'
WARNING:root:[Errno 2] No such file or directory: '/home/search/apps/jupyter/opt/samples/dhgate_full/2_validate0_pass/019029004'
WARNING:root:[Errno 2] No such file or directory: '/home/search/apps/jupyter/opt/samples/dhgate_full/2_validate0_pass/019030011001' 

之所以造成这个错误的原因就是因为base_path后面是已'/'结尾的导致的 re在os.path.join(src,dsr)得到了一个这么奇怪的结果
例子
import re,os
p =  '/home/search/apps/jupyter/opt/samples/dhgate_full'
s = '/home/search/apps/jupyter/opt/samples/dhgate_full/0_pass/002312/d213/3213.jpg','/home/search/apps/jupyter/opt/samples/dhgate_full/1_infringement/002312/d213/3213.jpg'
classes1,classes2 = '0_pass', '1_infringement'
for i in s:
    x = re.sub(r'(%s)'%p,os.path.join(p,'va'),i)
    print(x)
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
import re,os
p =  '/home/search/apps/jupyter/opt/samples/dhgate_full/'
s = '/home/search/apps/jupyter/opt/samples/dhgate_full/0_pass/002312/d213/3213.jpg','/home/search/apps/jupyter/opt/samples/dhgate_full/1_infringement/002312/d213/3213.jpg'
classes1,classes2 = '0_pass', '1_infringement'
for i in s:
    x = re.sub(r'(%s)'%p,os.path.join(p,'va'),i)
    print(x)
''' 

In [10]:
os.path.join(p,'v')

'/home/search/apps/jupyter/opt/samples/dhgate_full/v'

In [21]:
#val_list[1]
s = '/home/search/apps/jupyter/opt/samples/dhgate_full/0_pass/104006010/415779747.jpg'
re_name  = re.sub(r'(%s)'%(path),os.path.join(path,mv_file_name),s)#注意这里，我要在修改的基础是有全局path这个变量的
re_name_file = os.path.split(re_name)[0]#只要文件夹，不要文件名
re_name_file

'/home/search/apps/jupyter/opt/samples/dhgate_full/2_validate/0_pass/104006010'

In [24]:
import math
ratio = 0.1
validate_num = math.ceil(len(pass_0)*0.1)
np.random.seed(2018)
val_pass_0 = np.random.choice(pass_0,validate_num,replace=False).tolist()
val_infringement_1= np.random.choice(infringement_1,validate_num,replace=False).tolist()
val_list = val_pass_0 + val_infringement_1
#np.random.shuffle(val_list)
assert len(val_list) == validate_num*2

classes1,classes2 = '0_pass', '1_infringement'
path = '/home/search/apps/jupyter/opt/samples/dhgate_full'
logging.info('random to choice %s pictures for every class'%validate_num)#numpy array 不能用%d
mv_file_name = '2_validate'#4_surplus_data_0_pass
for i in tqdm(val_list):
    try:
        #pass_0.remove(i)#这个会严重拖延速度因为对应元素找list会是O（N）的复杂度
        #如果改成deque用线程也是没办法对应元素删除
        #方案1 用shuffle 然后直接把做切片，删除切片区间的路径
        #方案2 通过用label 一边迭代一边删除，但这个方法是不正确的，因为在没有迭代出label
        #类的总个数以及和其他类别的个数是无法知道要多了多少个path需要移动
        re_name  = re.sub(r'(%s)'%(path),os.path.join(path,mv_file_name),i)#注意这里，我要在修改的基础是有全局path这个变量的
        re_name_file = os.path.split(re_name)[0]#只要文件夹，不要文件名
        #surplus_file = os.path.join(path, mv_file_name)
        if not os.path.exists(re_name_file):
            os.makedirs(re_name_file)
        shutil.move(i,re_name_file)
        logging.info('validate file removed done')
        continue
    except Exception as e:
        logging.warning(e)

validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
v

validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
v

validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
v

validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
v

validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
v

...

validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
v

validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
v

validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
v

validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
v

validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
validate file removed done
v

In [9]:
import re,os
p =  '/home/search/apps/jupyter/opt/samples/dhgate_full'
s = '/home/search/apps/jupyter/opt/samples/dhgate_full/0_pass/002312/d213/3213.jpg','/home/search/apps/jupyter/opt/samples/dhgate_full/1_infringement/002312/d213/3213.jpg'
classes1,classes2 = '0_pass', '1_infringement'
for i in s:
    x = re.sub(r'(%s)'%p,os.path.join(p,'va'),i)
    print(x)

/home/search/apps/jupyter/opt/samples/dhgate_full/va/0_pass/002312/d213/3213.jpg
/home/search/apps/jupyter/opt/samples/dhgate_full/va/1_infringement/002312/d213/3213.jpg


In [ ]:
import random
a = ['dadadadad','dd','ddd']
np.random.seed(2018)
np.random.shuffle(a)
a

In [37]:
val_path = '/home/search/apps/jupyter/opt/samples/dhgate_full/2_validate'
val = [os.path.join(r,f) for r,ds,fs in os.walk(val_path) for f in fs if f.lower().endswith('jpg')or f.lower().endswith('jpeg')]
len(val)/2

42376.0

In [ ]:
def check_num(file_list,klass,auto_check=True,seed=20180123):
    import numpy as np
    import re,logging
    from tqdm import tqdm_notebook as tqdm
    path = '/home/search/apps/jupyter/opt/samples/dhgate_full/'
    classes1,classes2 = '0_pass', '1_infringement'
    pass_0, infringement_1 = [], []#deque(), deque()
    for cartegori in file_list:
        if classes1 in cartegori:
            pass_0.append(cartegori)
        elif classes2 in cartegori:
            infringement_1.append(cartegori)
    print(f'0_pass的数量有 {len(pass_0)} 张;',
          f'1_fringement的数量有 {len(infringement_1)} 张;',
          f'\t总共属于2类别的有{len(infringement_1)+len(pass_0)}张.',
         sep='\n\t')
    
  
    np.random.seed(seed)
    #if auto_check=True:
    surplus_num = len(pass_0)-len(infringement_1)
    big_file_name = max()
    logging.info('the surplus class have %s pictures'%surplus_num)#numpy array 不能用%d
    surplus_random = np.random.choice(pass_0,surplus_num,replace=False)
    assert len(surplus_random)==len(set(surplus_random))
    logging.info('random to choice surplus  %s pictures'%surplus_random)#numpy array 不能用%d
    mv_file_name = 'surplus_data'+str(big_file_name)
    for i in tqdm(surplus_random):
        try:
            #pass_0.remove(i)
            re_name  = re.sub(r'(%s|%s)'%(classes1,classes2),mv_file_name,i)
            re_name_file = os.path.split(re_name)[0]
            #surplus_file = os.path.join(path, mv_file_name)
            if not os.path.exist(re_name_file):
                os.mkdir(re_name_file)
            shutil.move(i,re_name_file)
            print('surplus file %s removed done'%surplus_num)
            print('0_pass have %s pictures'%pass_0)
        except Exception as e:
            logging.warning(e)

In [ ]:
import collections 
import threading 
import time 
candle=collections.deque(range(5)) 
def burn(direction,nextSource): 
    while True: 
        try: 
            next=nextSource() 
        except IndexError: 
            break 
        else: 
            print ('%s : %s' % (direction,next))
            time.sleep(0.1) 
            print ("done %s" % direction )
    return 
left=threading.Thread(target=burn,args=('left',candle.popleft)) 
right=threading.Thread(target=burn,args=('right',candle.pop)) 
left.start() 
right.start() 
left.join() 
right.join()



In [ ]:
print(f'修复后的图片directory 有 {len(file_list)}张 ,保存在变量名 file_list 当中;',
      f'没有字节的图片有{len(none_size)}张,保存在变量名 none_size 当中;',
      f'\t不能打开的图片有{len(verify)}张.保存在变量名 verify当中;',
      f'\t\t转换失败的有{len(convert_error)}张,保存在变量名 convert_error 当中;',
      f'\t\t\t 最终预处理不能的图片有{len(total_bad)}张,保存在变量名 total_bad 当中.',sep='\n\t')
verify == convert_error ==total_bad

In [ ]:
from collections import deque
pass_0, infringement_1 = deque(), deque()
for cartegori in file_list:
    if classes[0] in cartegori:
        pass_0.append(cartegori)
    elif classes[1] in cartegori:
        infringement_1.append(cartegori)
print(f'0_pass的数量有 {len(pass_0)} 张;',
      f'1_fringement的数量有 {len(infringement_1)} 张;',
      f'\t总共属于2类别的有{len(infringement_1)+len(pass_0)}张.',
     sep='\n\t')

In [ ]:
class FLAGS():
    pass
FLAGS.min_file_size = 1
FLAGS.check_image_format = True
FLAGS.MAX_NUM_IMAGES_PER_CLASS = 2 ** 27 - 1  # ~134M
base_path = '/home/search/apps/jupyter/opt/samples/dhgate_full'
val_path = '/home/search/apps/jupyter/opt/samples/dhgate_full/2_validate'

if FLAGS.check_image_format:
    def process(target_path, klass, autoklass=False,):
        import logging, imghdr, os, shutil, piexif, cv2
        from PIL import Image
        from tqdm import tqdm_notebook as tqdm
        from collections import deque,OrderedDict
        result = OrderedDict()
        path = target_path
        if not os.path.exists(path):
            logging.error("Image directory '" + path + "' not found.")
            return None
        if autoklass is False:
            klass = tuple(klass) if klass is not tuple else klass
            file_list = [os.path.join(r,f)for label in klass for r,ds, fs in sorted(os.walk(os.path.join(path,label))) 
                        for f in fs if f.lower().endswith('jpg') or f.lower().endswith('jpeg')]
        else:
            #klass = os.listdir(path)
            file_list = [os.path.join(r,f)for r,ds, fs in sorted(os.walk(path)) 
                        for f in fs if f.lower().endswith('jpg') or f.lower().endswith('jpeg')]


        #     classes_label = len(klass)


        #     for cartegori in pic_dirs:
        #         if classes[0] in cartegori:a
        #             pass_0.append(cartegori)
        #         elif classes[1] in cartegori:
        #             infringement_1.append(cartegori)

        #     logging.info(f'''0_pass的数量有 {len(pass_0)} 张,
        #     \t1_fringement的数量有 {len(infringement_1)} 张,
        #     \t\t总共属于{len(classes)}个类别,合计有{len(infringement_1)+len(pass_0)} 张.''')

        none_size, verify, convert_error, total_bad = deque(), deque(),deque(), deque()
        logging.info('start check image format')
        for num, dirs in enumerate(tqdm(file_list)):
            try:
                open_format = 'None'
                getsize = os.path.getsize(dirs)
                if getsize < FLAGS.min_file_size:   
                    none_size.append(dirs)
                    none_size_dir = os.path.join(path,"NONE_SIZE")
                    if not os.exists(none_size_dir): 
                        os.makedirs(none_size_dir)
                        logging.info('Make the the %s dircetory that DHgate NONE_SIZE file in your path directory '%none_size_dir)
                    file_list.remove(img)
                    logging.info(f'Remove {dirs}')
                    logging.warning('%s too small file size %d ,the NO.%s removed' % (dirs, getsize,num))
                    continue#直接略过开下个dirs
                    
                img_format = str(imghdr.what(dirs)) 
                if img_format == 'jpeg' or img_format == 'png' or img_format == 'gif':
                    continue
                else:
                    #cv_check = cv2.imread(dirs)
                    img_open = Image.open(dirs)
                    x, y = img_open.size
                    mode = img_open.mode
                    open_format = img_open.format
                    call_verify = img_open.verify()
                    if not call_verify:
                        if x < 299 or y < 299:
                            file_list.remove(dirs)
                            #os.remove(img)
                            small_size_dir  = os.path.join(path, 'SMALL_SIZE')
                            if not os.path.exists(small_size_dir):
                                os.makedirs(small_size_dir)
                                logging.info('Make the the %s dircetory that DHgate SMALL_SIZE file in your path directory '%small_size_dir)
                            shutil.move(dirs, small_size_dir)
                            logging.warning(
                            '%s too small img size %dx%d , the NO.%s removed to SMALL_SIZE file' % (dirs, img_open.size[0], img_open.size[1], num))
                            continue
                            #                 else:
                            #                     verify.append(dirs)
                            #                     logging.warning('%s check verify the NO.%s Invalid image'%(dirs,num))

                        if open_format != 'JPEG':
                            piexif.remove(dirs)
                            other_mode_dir = os.path.join(path, 'OTHER_MODE_CONVERT')
                            if not os.path.exists(other_mode_dir):
                                os.makedirs(other_mode_dir)
                                logging.info('Make the %s dircetory that DHgate OTHER_MODE_CONVERT file in your path directory '%other_mode_dir)
                            tmp_img = other_mode_dir +os.sep+os.path.basename(dirs).split('.')[0]+ '.jpg'#或者直接str（num)
                            if mode == 'RGB' or mode == 'RGBX':
                                img_open.convert('RGB')
                                img_open.save(tmp_img)
                            else:
                                image_new = Image.new("RGB", img_open.size, (255, 255, 255))
                                image_new.paste(img_open, img_open)
                                image_new.save(tmp_img)

                            if os.path.exists(tmp_img) and os.path.getsize(tmp_img) > 0:
                                shutil.move(tmp_img, dirs)
                                logging.info('%s %s convert to the NO.%s jpeg success' % (dirs, open_format, num))
                            else:
                                logging.warning('%s %s convert to the NO.%s jpeg error' % (dirs, open_format, num))
                                file_list.remove(dirs)
                                convert_error.append(dirs)
                        continue#verity
                    else:
                        verify.append(dirs)
                        logging.warning('%s check verify the NO.%s Invalid image'%(dirs,num))

            except (IOError, ValueError) as err:
                #file_list.remove(dirs)
                #os.remove(img)
                bad_path = os.path.join(path, open_format)
                if not os.path.exists(bad_path):
                    os.makedirs(bad_path)
                    logging.info('Make the %s directory that DHgate the %s file in your path directory '%(bad_path, open_format))
                total_bad.append(dirs)
                shutil.move(dirs, bad_path)
                logging.warning('%s %s error : %s, removed' % (dirs, open_format, err, num))
                continue

        if len(file_list) < 20:
            logging.warning('Folder ' + ' only has ' + str(len(file_list))+ ' less than 20 images, skiped')
            #continue 在labbel中迭代才需要
        elif len(file_list) > FLAGS.MAX_NUM_IMAGES_PER_CLASS:
            tf.logging.warning(
                'Folder {} has more than {} images. Some images will '
                'never be selected.'.format(os.path.join(image_dir, label), MAX_NUM_IMAGES_PER_CLASS))
        return   file_list, none_size, verify, convert_error, total_bad

    
    
path = '/home/search/apps/jupyter/opt/samples/dhgate_full'
classes = '0_pass', '1_infringement'
file_list, none_size, verify, convert_error, total_bad = process(path,classes)
print(f'修复后的图片directory 有 {len(file_list)}张 ,保存在变量名 file_list 当中;',
      f'没有字节的图片有{len(none_size)}张,保存在变量名 none_size 当中;',
      f'\t不能打开的图片有{len(verify)}张.保存在变量名 verify当中;',
      f'\t\t转换失败的有{len(convert_error)}张,保存在变量名 convert_error 当中;',
      f'\t\t\t 最终预处理不能的图片有{len(total_bad)}张,保存在变量名 total_bad 当中.',sep='\n\t')

pass_0, infringement_1 = deque(), deque()
for cartegori in file_list:
    if classes[0] in cartegori:
        pass_0.append(cartegori)
    elif classes[1] in cartegori:
        infringement_1.append(cartegori)
print(f'0_pass的数量有 {len(pass_0)} 张;',
      f'1_fringement的数量有 {len(infringement_1)} 张;',
      f'\t总共属于2类别的有{len(infringement_1)+len(pass_0)}张.',
     sep='\n\t')

In [ ]:
'''概念一样但是检测更细致的附加方案'''

In [ ]:
class FLAGS():
    pass
FLAGS.check_image_format = True
FLAGS.MAX_NUM_IMAGES_PER_CLASS = 2 ** 27 - 1  # ~134M
base_path = '/home/search/apps/jupyter/opt/samples/dhgate_full'
val_path = '/home/search/apps/jupyter/opt/samples/dhgate_full/2_validate'

if FLAGS.check_image_format:

    def process2(target_path, klass, autoklass=False,):
        import logging, imghdr, os, shutil, piexif, cv2
        from PIL import Image
        from tqdm import tqdm_notebook as tqdm
        from collections import deque

        path = target_path
        if not os.path.exists(path):
            logging.error("Image directory '" + path + "' not found.")
            return None
        if autoklass is False:
            klass = tuple(klass) if klass is not tuple else klass
            file_list = [os.path.join(r,f)for label in klass for r,ds, fs in sorted(os.walk(os.path.join(path,label))) 
                        for f in fs if f.lower().endswith('jpg') or f.lower().endswith('jpeg')]
        else:
            #klass = os.listdir(path)
            file_list = [os.path.join(r,f)for r,ds, fs in sorted(os.walk(path)) 
                        for f in fs if f.lower().endswith('jpg') or f.lower().endswith('jpeg')]


    #     classes_label = len(klass)


    #     for cartegori in pic_dirs:
    #         if classes[0] in cartegori:
    #             pass_0.append(cartegori)
    #         elif classes[1] in cartegori:
    #             infringement_1.append(cartegori)

    #     logging.info(f'''0_pass的数量有 {len(pass_0)} 张,
    #     \t1_fringement的数量有 {len(infringement_1)} 张,
    #     \t\t总共属于{len(classes)}个类别,合计有{len(infringement_1)+len(pass_0)} 张.''')

        none_size, verify, convert_error, total_bad = [], [], [], []
        logging.info('start check image format')
        for num, dirs in enumerate(tqdm(file_list)):
            try:
                open_format = 'None'
                getsize = os.path.getsize(dirs)
                if getsize < FLAGS.min_file_size:
                    none_size.append(dirs)
                    none_size_dir = os.path.join(path,"NONE_SIZE")
                    if not os.exists(none_size_dir): 
                        os.makedirs(none_size_dir)
                        logging.info('Make the the %s dircetory that DHgate NONE_SIZE file in your path directory '%none_size_dir)
                    file_list.remove(img)
                    logging.info(f'Remove {dirs}')
                    logging.warning('%s too small file size %d ,the NO.%s removed' % (dirs, getsize,num))
                    continue#直接略过开下个dirs
                try:
                    img_open = Image.open(dirs)
                    call_verify = img_open.verify()
                    if call_verify is None:
                        #shutil.rmtree(dirs)
                        #Additionally, you might want to also check that the Dhgate images are actually a JPG by looking at the image format
                        #通过查看图像格式来检查图像实际上是JPG：
                        img_format = str(imghdr.what(dirs)) 
                        if img_format == 'jpeg' or img_format == 'png' or img_format == 'gif':
                            continue
                        else:
                            #cv_check = cv2.imread(dirs)
                            #img_open = Image.open(dirs)
                            x, y = img_open.size
                            mode = img_open.mode
                            open_format = img_open.format

                            if x < 299 or y < 299:
                                file_list.remove(dirs)
                                #os.remove(img)
                                small_size_dir  = os.path.join(path, 'SMALL_SIZE')
                                if not os.path.exists(small_size_dir):
                                    os.makedirs(small_size_dir)
                                    logging.info('Make the DHgate SMALL_SIZE file in your path directory ')
                                shutil.move(dirs, small_size_dir)
                                logging.warning(
                                '%s too small img size %dx%d , the NO.%s removed to SMALL_SIZE file' % (dirs, img_open.size[0], img_open.size[1], num))
                                continue

                            if open_format != 'JPEG':
                                piexif.remove(dirs)
                                other_mode_dir = os.path.join(path, 'OTHER_MODE')
                                if not os.path.exists(other_mode_dir):
                                    os.makedirs(other_mode_dir)
                                tmp_img = other_mode_dir +os.sep+os.path.basename(dirs).split('.')[0]+ '.jpg'#或者直接str（num)
                                if mode == 'RGB' or mode == 'RGBX':
                                    img_open.convert('RGB')
                                    img_open.save(tmp_img)
                                else:
                                    image_new = Image.new("RGB", img_open.size, (255, 255, 255))
                                    image_new.paste(img_open, img_open)
                                    image_new.save(tmp_img)

                                if os.path.exists(tmp_img) and os.path.getsize(tmp_img) > 0:
                                    shutil.move(tmp_img, dirs)
                                    logging.info('%s %s convert to the NO.%s jpeg success' % (dirs, open_format, num))
                                else:
                                    logging.warning('%s %s convert to the NO.%s jpeg error' % (dirs, open_format, num))
                                    file_list.remove(dirs)
                                    convert_error.append(dirs)
                        continue
                    else:
                        verify.append(dirs)
                        logging.info('%s the NO.%s jpeg error' % (dirs, open_format, num))
                except Exception as e:
                    logging.warning('%s %s convert to the NO.%s jpeg error' % (dirs, open_format, num))

            except (IOError, ValueError) as err:
                #file_list.remove(dirs)
                #os.remove(img)
                bad_path = os.path.join(path, mode)
                if not os.path.exists(bad_path):
                    os.makedirs(bad_path)    
                total_bad.append(dirs)
                shutil.move(dirs, bad_path)
                logging.warning('%s %s error : %s, removed' % (dirs, open_format, err, num))
                continue

        if len(file_list) < 20:
            logging.warning(
                'Folder ' + ' only has ' + str(len(file_list))
                + ' less than 20 images, skiped')
        elif len(file_list) > FLAGS.MAX_NUM_IMAGES_PER_CLASS:
            tf.logging.warning(
                'Folder {} has more than {} images. Some images will '
                'never be selected.'.format(os.path.join(image_dir, label), MAX_NUM_IMAGES_PER_CLASS))
        return   file_list, none_size, verify, convert_error, total_bad
            #continue
    # elif len(file_list) > MAX_NUM_IMAGES_PER_CLASS:
    #     tf.logging.warning(
    #         'Folder {} has more than {} images. Some images will '
    #         'never be selected.'.format(os.path.join(image_dir, label), MAX_NUM_IMAGES_PER_CLASS))


In [ ]:
'''3层保险的验证方案'''

In [ ]:
def process3(file_list):
    from PIL import Image
    from tqdm import tqdm_notebook as tqdm
    from collections import deque
    import piexif
    Invalid_pic = []
    # suppose im_path is a valid image path
    val_path = '/home/search/apps/jupyter/opt/samples/dhgate_full/Validate'
    png_path = '/home/search/apps/jupyter/opt/samples/dhgate_full/PNG'
    mpo_path = '/home/search/apps/jupyter/opt/samples/dhgate_full/MPO'

    for dirs in tqdm(file_list):
        try:
            piexif.remove(dirs)#删掉脏数据中的个人信息，那些用手机或相机拍的带有个人信息的照片
        except Exception as e:
            print(e,dirs)
            piexif_bad.append(dirs)
            file_list.remove(dirs)
            continue
        try:
            img = Image.open(dirs)
        except IOError: 
            file_list.remove(dirs)
            print(dirs)
            try:
                shutil.move(dirs, val_path)
            except Exception as e:
                continue
        try:
            img.verify()
            #print('Valid image')
        except Exception as e:
            print('Invalid image',e,dirs)
            Invalid_pic.append(dirs)
            file_list.remove(dirs)
            shutil.move(dirs, val_path)
            #shutil.rmtree(dirs)
            #Additionally, you might want to also check that the Dhgate images are actually a JPG by looking at the image format
            #通过查看图像格式来检查图像实际上是JPG：
            continue
        try:
            if img.format == 'JPEG':
                #print('JPEG image')
                continue
            elif img.format =='PNG':
                    shutil.move(dirs, png_path)
                    print('Invalid image type',dirs)
            else:
                    shutil.move(dirs, mpo_path)
        #         Invalid_pic.append(dirs)
        #         Invalid_pic_list.append(dirs)
                    print('Invalid image type',dirs)
        except Exception as e:
            continue
    return Invalid_pic



In [ ]:
Invalid_pic = process3(file_list)

In [39]:
8361/60

139.35

In [40]:
8361/3600

2.3225